# Traffic Accidents in Brazillian Federal Highways

In [1]:
import mysql.connector as mysql
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [32]:
db = mysql.connect(
    host = "us-cdbr-east-04.cleardb.com",
    user = "bd7bbf83ab7643",
    password = "dfce7117",
    database='heroku_aba902d59bebc6b'
)

In [33]:
cursor = db.cursor()

In [24]:
cursor.execute("CREATE DATABASE brazil_traffic,")

OperationalError: 2013 (HY000): Lost connection to MySQL server during query

In [24]:
unique_dict = {}


Index(['id', 'pesid', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br',
       'km', 'municipio', 'causa_principal', 'causa_acidente',
       'ordem_tipo_acidente', 'tipo_acidente', 'classificacao_acidente',
       'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista',
       'tracado_via', 'uso_solo', 'id_veiculo', 'tipo_veiculo', 'marca',
       'ano_fabricacao_veiculo', 'tipo_envolvido', 'estado_fisico', 'idade',
       'sexo', 'ilesos', 'feridos_leves', 'feridos_graves', 'mortos',
       'latitude', 'longitude', 'regional', 'delegacia', 'uop'],
      dtype='object')

In [34]:
cursor.execute("""
CREATE TABLE traffic_accidents_br(
    id INT(50) NOT NULL AUTO_INCREMENT,
    pesid INT(50),
    data_inversa DATE,
    dia_semana ENUM('segunda-feira','terça-feira','quarta-feira','quinta-feira','sexta-feira','sábado','domingo'),
    horario TIME(0),
    uf ENUM('PR','SC','GO','PB','TO','RS','PE','PI','CE','MS','SP','BA','SE','DF','RJ','MG','PA','RO','AL','RN','MT','ES','MA','AC','RR','AM','AP'),
    br VARCHAR(3),
    km FLOAT(4,2),
    municipio VARCHAR(30),
    causa_principal ENUM('Sim','Não'),
    causa_acidente ENUM('Fenômenos da Natureza','Falta de Atenção à Condução','Animais na Pista','Avarias e/ou desgaste excessivo no pneu','Ingestão de Álcool','Não guardar distância de segurança','Defeito Mecânico no Veículo','Desobediência às normas de trânsito pelo condutor','Velocidade Incompatível','Restrição de Visibilidade','Falta de Atenção do Pedestre','Condutor Dormindo','Mal Súbito','Pista Escorregadia','Ultrapassagem Indevida','Defeito na Via','Sinalização da via insuficiente ou inadequada','Carga excessiva e/ou mal acondicionada','Objeto estático sobre o leito carroçável','Deficiência ou não Acionamento do Sistema de Iluminação/Sinalização do Veículo','Ingestão de Substâncias Psicoativas','Agressão Externa','Desobediência às normas de trânsito pelo pedestre'),
    ordem_tipo_acidente ENUM(1,2,3),
    ordem_tipo_acidente ENUM(1,2,3,4,5,6,7,8,9),
    tipo_acidente ENUM('Queda de ocupante de veículo','Colisão com objeto estático','Capotamento','Tombamento','Saída de leito carroçável','Colisão traseira','Atropelamento de Animal','Colisão frontal','Incêndio','Atropelamento de Pedestre','Colisão lateral','Colisão transversal','Derramamento de carga','Colisão com objeto em movimento','Engavetamento','Danos eventuais'),
    classificacao_acidente ENUM('Com Vítimas Feridas','Sem Vítimas','Com Vítimas Fatais'),
    fase_dia ENUM('Plena Noite', 'Amanhecer', 'Pleno dia', 'Anoitecer'),
    sentido_via ENUM('Crescente', 'Decrescente', 'Não Informado'),
    condicao_metereologica ENUM('Chuva','Garoa/Chuvisco','Céu Claro','Nublado','Ignorado','Nevoeiro/Neblina','Sol','Vento','Granizo','Neve'),
    tipo_pista ENUM('Simples', 'Dupla', 'Múltipla'),
    tracado_via ENUM('Reta','Curva','Não Informado','Interseção de vias','Rotatória','Desvio Temporário','Viaduto','Ponte','Retorno Regulamentado','Túnel'),
    uso_solo ENUM('Não','Sim'),
    id_veiculo INT(50),
    tipo_veiculo ENUM('Motocicleta','Automóvel','Motoneta','Caminhão','Caminhonete','Utilitário','Camioneta','Caminhão-trator','Carroça-charrete','Bicicleta','Semireboque','Reboque','Micro-ônibus','Ônibus','Outros','Ciclomotor','Trator de rodas','Não Informado','Trator misto','Carro de mão','Trem-bonde','Triciclo','Trator de esteira','Chassi-plataforma','Quadriciclo'),
    marca VARCHAR(60),
    ano_fabricacao INT(4),
    tipo_envolvido ENUM('Condutor','Passageiro','Pedestre','Testemunha','Não Informado','Cavaleiro'),
    estado_fisico ENUM('Lesões Graves','Ileso','Lesões Leves','Óbito','Não Informado'),
    idade INT(3),
    sexo ENUM('Masculino', 'Feminino', 'Não Informado', 'Ignorado'),
    ilesos INT(2),
    feridos_leves INT(2),
    feridos_graves INT(2),
    mortos INT(2),
    latitude VARCHAR(60),
    longitude VARCHAR(60),
    regional VARCHAR(5),
    delegacia VARCHAR(10),
    uop VARCHAR(10),
    PRIMARY KEY (id),
    );
""")

ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '1,2,3),
    ordem_tipo_acidente ENUM(1,2,3,4,5,6,7,8,9),
    tipo_acidente ENUM(' at line 13